### Fonction qui implémente le problème maître restreint (de la génération de colonnes)

Le problème maître restreint, qui est la reformulation Dantzig-Wolfe du MIP est :

max $\sum_{c \in C_L'} w_c z_c$

tel que :
- $\sum_{c \in C_L'(i)} z_c \le 1, \forall i \in V$,
- $z \in [0,1]^{|C_L'|}$,

où $C_L' \subset C_L$.

On note $z_c=\{0,1\}=1$ si le cycle $c$ est choisi.

On réécrit le problème en le mettant sous forme de minimisation :

min $- \sum_{c \in C_L'} w_c z_c$

tel que :
- $\sum_{c \in C_L'(i)} z_c \le 1, \forall i \in V$,
- $z \in [0,1]^{|C_L'|}$,

où $C_L' \subset C_L$.

In [2]:
function node_master(node_pool)
    # Node_pool : contient tous les patterns d'un noeud
    # node_pool[i] : 1 pattern, situé à l'indice i
    
    master = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(master, "OutputFlag", 0)
    
    # Variable ylc
    # Le premier indice est l'indice l (cf klimentova), le 2e est l'indice du cycle dans le sous-graphe l,  
    ylc = Array{Array{VariableRef,1},1}(undef,nv(new_inst.graph))
    for l in 1:nv(new_inst.graph)
        ylc[l]=Array{VariableRef,1}(undef,size(node_pool[l],1))
    end
    for l in 1:nv(new_inst.graph)
        for c in 1:size(node_pool[l],1)
            ylc[l][c]=@variable(master,lower_bound=0)
            set_name(ylc[l][c], "ylc_$l,$c")
        end
    end
    
    # Contraintes
    # Chaque sommet apparait au maximum une fois dans chaque cycle/chaque pattern
    sommets=@constraint(master,sommet[ind in 1:nv(new_inst.graph)],sum(ylc[l][c] for l in 1:nv(new_inst.graph), c in 1:size(node_pool[l],1) if node_pool[l][c][ind+1]>0) <=1)
    
    # Objectif : maximiser le nombre de transplantations
    @objective(master,Max,sum(ylc[l][c]*sum(node_pool[l][c][i] for i in 2:size(node_pool[l][c],1))
            for l in 1:nv(new_inst.graph),c in 1:size(node_pool[l],1)))
    
    # Retourne les contraintes, les variables de référence et le modèle
    return sommets,ylc,master
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[2]:12